# Introducción a la Inferencia Causal

# Clase 3: métodos para estimar efectos causales a partir de datos observacionales 

Queremos ver el efecto de X en Y. Pero **no podemos:**

*   **Experimentar con X.** En general esto sucede cuando queremos estudiar algunas decisiones de vida de las personas, como por ejemplo: asignar años de educación a las personas, obligar a ir o no ir al servicio militar.
*   **Controlar por todas las variables necesarias.** En general porque no conocemos el verdadero modelo que genera los datos y no estaremos seguros de qué variables incluir o no. 

¿Qué hacemos?

Desde la econometría se han pensado algunas soluciones, que veremos en la clase de hoy: variables instrumentales, diferencias en diferencias y regresiones con datos en panel, regresión discontinua, control sintético... Profundizaremos en las primeras dos.

El objetivo de todas ellas es la misma: tratar de construir un _buen contrafáctico_, donde el _supuesto de identificación_ (aquello que debemos suponer que se cumple para poder asegurar que estamos encontrando un efecto causal, no testeable empíricamente) sea _creíble_.

## Librerías necesarias

* [statsmodels](https://www.statsmodels.org/stable/index.html): regresión lineal, con lindas tablas de inferencia estadística.

* [linearmodels](https://pypi.org/project/linearmodels/): funciona como extensión de statsmodels incorporando modelos específicos de inferencia causal, como regresión en paneles, variables instrumentales y otros.

In [ ]:
# Instalar
%pip install linearmodels --quiet

# Importar

import pandas as pd
import numpy as np
import io

from google.colab import files

import matplotlib.pyplot as plt
import graphviz as gr

import statsmodels.formula.api as smf
from linearmodels import IV2SLS  # módulo específico de variables instrumentales


## **Variables Instrumentales (VI)**

Preguntas como ¿mayor eduación mejora los salarios? ¿las instituciones coloniales afectaron el desarrollo económico posterior? ¿ir a la colimba en Argentina incrementa las chances de cometer crímenes luego? han sido respondidas usando variables instrumentales.

Pensemos por ejemplo en la primera, que es lo que quisieron responder [Angrist y Krueguer (1991)]() en uno de los trabajos pioneros en usar VI. 

**Ejercicio de lectura activa**. Supongamos que tenemos datos de años en el sistema educativo y de salarios. ¿Cuál sería el problema de regresar los salarios en los años de educación para tratar de estimar el efecto de la educación en los salarios?

In [ ]:
# Mi respuesta aquí.

print("")

**Respuesta**.

Podría estar pasando algo así:



In [ ]:
g = gr.Digraph()

g.edge("motivación", "educ")
g.edge("motivación", "salario")
g.edge("educ", "salario")
g

**Ejercicio de lectura activa**: ¿qué significa ese grafo? ¿Por qué decimos que no podríamos identificar correctamente el efecto causal si esa fuera la situación?

O algo así:

In [ ]:
g = gr.Digraph()

g.edge("historia familiar", "educ")
g.edge("historia familiar", "salario")
g.edge("educ", "salario")
g

**Ejercicio de lectura activa**: ¿qué significa ese grafo? ¿Por qué decimos que no podríamos identificar correctamente el efecto causal si esa fuera la situación?

Y lo más probable es que no podamos medir adecuadamente la motivación o la historia familiar. En general contamos con datos más limitados. Entonces, controlar por los posibles confusores no es una opción. Teníamos otra opción: experimentar. Pero asignar aleatoriamente la motivación o la historia familiar está complicado...


Acá entra a jugar Variables Instrumentales. Este método lo que busca es quedarse con la parte de la variabilidad de $educ$ que sea **exógena**, esto es, la parte de tener más años de educación que afecta al salario por la educación en sí, y no por alguna variable confusora omitida que correlaciona con la educación y también con el salario (como la motivación).

Para eso, utiliza una tercer variable a modo de **instrumento** (que notaremos con $Z$). Coloquialmente, un instrumento es **una variable que permita hacer variar a la educación sin que varíe ninguna otra posible variable omitida**. Es parecido a experimentar: hacer variar al tratamiento pero que el resto de las características de tratados y no tratados se mantengan iguales.

En términos formales, un instrumento es una variable que cumple con dos condiciones:

* **Relevancia**: $Cov(T, Z) \neq 0$. El INSTRUMENTO está correlacionado con T. Esta condición es **testeable**.
* **Validez**, también conocida como **restricción de exclusión**:  el INSTRUMENTO sólo afecta a Y a través de T. Esto implica que no está correlacionado directamente con $Y$, ni con las variables omitidas. Es decir, si $Y = \beta_0 + \beta_1 T + u$, esta condición significa que $Cov(u, Z) = 0$. Esta condición no se puede testear, es el **supuesto de identificación**, y sólo podemos argumentar que se cumple teóricamente.

Es decir, se ve así:






In [ ]:
g = gr.Digraph()

g.edge("motivación", "educ")
g.edge("motivación", "salario")
g.edge("educ", "salario")
g.edge("INSTRUMENTO", "educ")
g


<center>
<img src="https://storage.googleapis.com/humai-datasets/imagenes/causalidad/3_MetodosDeEstimacion/img1_instrum.png" style="margin:auto" height=200/>
</center>



### ¿Cómo funciona variables instrumentales?

Veamos tres formas de abordar cómo funciona:

**i. En términos coloquiales**

Podemos pensar que cuando no controlamos por confusores, la variabilidad de T que afecta a Y tiene dos partes:
  - Una que de la variabilidad se debe al "tratamiento en sí" o **variabilidad exógena**. En nuestro ejemplo, la parte de la educación que se deba a las capacidades o herramientas adquiridas en sí durante el tiempo en instituciones educativas. 
  - Otra parte de la variabilidad se debe a las variables confusoras omitidas o **variabilidad endógena**. En nuestro ejemplo, la parte de la eduación que se asocie a la motivación propia de las personas, y que si omitimos la variable motivación en la ecuación se va a mover cuando se mueva la educación.

Sabemos que el instrumento captura parte de la variabilidad exógena del T: sólo mueve a "la educación en sí", porque por definición en instrumento sólo afecta a Y a través de la "educación en sí". 

  ¿Cuál fue el instrumento elegido por Angrist y Krueguer? El trimestre de nacimiento. 
  
  
  **Ejercicio de lectura activa**: tratar de pensar por qué el trimestre de nacimiento cumple con las condiciones para ser un instrumento..


  **Respuesta**.  
  
  Los autores argumentan que el trimestre de nacimiento hace que algunas personas pasen más meses en instituciones educactivas que otras (es decir, el trimestre de nacimiento es **relevante** para la educación), pero que el trimestre de nacimiento no puede influir en el salario directamente (es decir,es **válido**). El trimestre de nacimiento sólo puede afectar al salario a través del tiempo de educación recibida.


Hay al menos dos formas de obtener el estimador de VI:


**ii. Mínimos Cuadrados en Dos Etapas   (o 2SLS, por _Two Stages Least Squares_)**

*   Primera etapa: regresar $T$ en el instrumento $Z$, es decir, estimar $T = \alpha_0 + \alpha_1 Z + v $ y obtener $\hat{T}$. ¡Esta es la parte de T que se asocia al instrumento! Es decir, permite captar la parte de la educación que varía cuando cambia el trimestre de nacimiento. Esa parte sólo se asocia a tener más tiempo en el sistema educativo y no a cualquier otro confusor.
*   Segunda etapa: regresar Y en $\hat{T}$, es decir, estimar $ Y =  \beta_0 + \beta_1 \hat{T} + u$.  Esa parte identifica el efecto causal, porque $\hat{T}$ es la parte de $T$ que varía por tener más años en el sistema educativo y no por algún otro confusor.

$\hat{\beta_1}$ será el estimador de variables instrumentales. Captura la parte de T que afecta a Y de manera exógena, a través del instrumento.


Estimemos ambas etapas con los datos de Angrist y Krueguer (1991):

In [ ]:
ak = pd.read_csv("https://storage.googleapis.com/humai-datasets/causalidad/ak91.txt")
ak .head()

Graficamos la relación entre el trimestre de nacimiento y los años de educación (una forma de mirar la relevancia!):


In [ ]:
# Graficamos la relación entre el trimestre de nacimiento y los años de educación (una forma de mirar la relevancia!)
group_data = (ak
              .groupby(["nacim_a", "nacim_trim"])
              [["log_salario", "educ"]]
              .mean()
              .reset_index()
              .assign(nacim_a_trim = lambda d: d["nacim_a"] + (d["nacim_trim"])/4))

plt.figure(figsize=(15,6))
plt.plot(group_data["nacim_a_trim"], group_data["educ"], zorder=-1)
for q in range(1, 5):
    x = group_data.query(f"nacim_trim=={q}")["nacim_a_trim"]
    y = group_data.query(f"nacim_trim=={q}")["educ"]
    plt.scatter(x, y, marker="s", s=200, c=f"C{q}")
    plt.scatter(x, y, marker=f"${q}$", s=100, c=f"white")

plt.title("Años de educación según trimestre de nacimiento")
plt.xlabel("Año-trimestre de nacimiento")
plt.ylabel("Años de educación");

Estimemos la primera etapa: regresemos los años de educación en el trimestre de nacimiento.

Para simplificar, creamos una variable binaria que es igual a 1 si la persona nació en el cuarto trimestre del año.

In [ ]:
# Creamos la variable binaria

ak["t4"] = [(x == 4)*1 for x in ak.nacim_trim] 

pd.crosstab(ak.nacim_trim, ak.t4)


In [ ]:
# Estimamos la primera etapa
etapa1 = smf.ols("educ ~  1 + t4", data=ak).fit()

print("estimador del parámetro de t4 ", etapa1.params["t4"])
print("p-valor del parámetro de t4: ", etapa1.pvalues["t4"])

¡La primera etapa sirve para testear la relevancia del instrumento! Acá vemos que nacer en el cuarto trimestre tiene una correlación positiva y significativa (el p-valor es casi cero) con los años de educación.

La segunda condición, la validez, no la podemos testear, es nuestro supuesto de identificación.


Estimemos la segunda etapa:

Primero debemos obtener $\hat{educ}$, las predicciones de los años de educación usando el instrumento, a partir de la regresión que hicimos en la primera etapa:

In [ ]:
# Los valores predichos para educ
ak["educfit"] = etapa1.fittedvalues

In [ ]:
# Estimamos la segunda etapa:
etapa2 = smf.ols("log_salario ~ 1+ educfit",
                     data = ak).fit()

etapa2.params["educfit"]

Los años de educación parecen tener un efecto positivo en el salario. PERO CUIDADO: **no** podemos usar los errores estándar (y por lo tanto los p-valores) de esta segunda etapa directamente para hacer inferencia respecto de si el efecto es significativo o no. Requieren una corrección. Estimar así a mano las dos etapas sólo sirve para obtener el estimador, y es una forma intuitiva de entender qué hace VI. Pero no se preocupen, a continuación veremos cómo estimarlo con la librería linearmodels para obtener el estimador directo, con los errores estándar y p-valores correctos.



_Nota_. Se pueden agregar variables de control para obtener un estimador más preciso (como el año o el estado de nacimiento de las personas), siempre que las agreguemos tanto en la primera como en las segunda etapa. 

**iii. Obtener el estimador de VI directamente**

_Esta es una derivación un poco más formal del estimador de VI._


Sin un instrumento estaríamos estimando:

$$Y = \beta_0 + \beta_1 T + u$$


Notemos que otra forma de expresar que se produce un sesgo cuando omitimos una variable confusora relevante como la motivación es afirmar que $Cov(T,u) \neq 0$. La variable confusora es una variable correlacionada con $Y$ que estamos omitiendo, por lo tanto está en $u$, y también sabemos que está correlacionada con $T$, por lo tanto  $Cov(T,u) \neq 0$.


Tomando la covarianza con el instrumento $Z$ obtenemos:

$$Cov(Y, Z) = Cov (\beta_0 + \beta T + u \; , \; Z) = \beta_1 Cov(T,Z) + Cov(u, Z)⇒ $$ 
 

$$\beta_1 = \frac{Cov(Y, Z)}{Cov(T, Z)} + \frac{Cov(u, Z)}{Cov(T, Z)}$$  


Notemos que las dos condiciones con las que debe cumplir el instrumento implican que $\frac{Cov(u, Z)}{Cov(T, Z)} =0$:

* Relevancia: $Cov(T, Z) \neq 0$

* Validez: $Cov(u, Z) = 0$

Por lo tanto, si se cumplen las condiciones para que $Z$ sea un instrumento y dividiendo por $V(Z)$:


$$\beta_1 = \frac{Cov(Y, Z)/ V(Z)}{Cov(T, Z)/V(Z)} = \frac{forma \; reducida}{primera \; etapa} $$  

$\hat{\beta_1}$ **es el estimador de varaibles instrumentales**, lo podemos obtener estimando esas covarianzas y varianzas.

Notemos que tanto el numerador como el denominador son coeficientes de regresión. 

- El numerador es el coeficiente de regresar $Y$ en $Z$. Es decir, el impacto que $Z$ tendrá en $Y$ (que sabemos que impacta solo a través de T). Se lo conoce como "el coeficiente de la forma reducida". 

- El denominador es el coeficiente de regresar $T$ en $Z$, es decir ¡el coeficiente de la primera etapa!

Intuitivamente, lo que significa esta fórmula es lo siguiente: ok, es difícil encontrar el efecto de $T$ en $Y$ porque hay variables omitidas. Pero fácilmente podemos encontrar el efecto de $Z$ en $Y$ (que sólo lo tiene a traves de $T$ y ninguna otra variable por la validez, no hay confusores omitidos). Pero como lo que nos interesa es el impacto de $T$ en $Y$, y no el impacto de $Z$ en $Y$, luego debemos escalar nuestra estimación por la relación entre $Z$ y $T$.



Si el instrumento es una variable binaria, el estimando de VI se simplifica aún más:

$$\beta_1 = \frac{E[Y \; | \; Z=1] - E[Y \; | \; Z=0] }{E[T \; | \; Z=1] - E[T \; | \; Z=0]} $$

A este estimador se lo suele llamar "el estimador de Wald".

Vamos a los datos de Angrist y Krueguer.

Primero estimemos la forma reducida (el impacto del trimestre de nacimiento en el salario):

In [ ]:
# Estimamos la forma reducida (regresar salarios en el cuarto trimestre de nacimiento)
reducida = smf.ols("log_salario ~ 1 + t4", data=ak).fit()

print("estimación: ", reducida.params["t4"])


Hay una relación positiva. Podemos graficarla:

In [ ]:
# Graficamos la forma reducida

plt.figure(figsize=(15,6))
plt.plot(group_data["nacim_a_trim"], group_data["log_salario"], zorder=-1)
for q in range(1, 5):
    x = group_data.query(f"nacim_trim=={q}")["nacim_a_trim"]
    y = group_data.query(f"nacim_trim=={q}")["log_salario"]
    plt.scatter(x, y, marker="s", s=200, c=f"C{q}")
    plt.scatter(x, y, marker=f"${q}$", s=100, c=f"white")

plt.title("Salario semanal según año y trimestre de nacimiento")
plt.xlabel("Año-trimestre de nacimiento")
plt.ylabel("Logaritmo del salario semanal");


Vemos que los que nacen en el cuarto trimestre suelen tener salarios más altos. Suponer validez del instrumento implica suponer que este impacto se produce a través de la educación: los nacidos en el cuarto trimestre tienen más años de educación y ganan más. Ahora bien a nosotros nos interesa la relación entre la educación y el salario, no del trimestre y el salario. Reescalemos por la relación entre el trimestre de nacmiento y la educación (por la primera etapa):

In [ ]:
# Podemos estimar VI "a mano" usando la fórmula de arriba
reducida.params["t4"] / etapa1.params["t4"]

Vemos que el valor del estimador de VI es mayor al estimador de la forma reducida.

**Ejercicio de lectura activa**: ¿por qué es mayor?

**Respuesta**. Es mayor porque la correlación entre el trimestre de nacimiento y los años de educación es positiva y menor a 1. Entonces es una cuestión de proporción. Cuando varía en 1 el trimestre de nacimiento, los años de educación varían en menos que 1. La forma reducida nos dice que cuando varíe en 1 el trimestre de nacimiento, varía en 0.6 el salario. Entonces, para que varíen en 1 los años de educación, el trimestre "está variando en más que 1", y por lo tanto el salario varía "en más que 0,6".


Ahora sí, estimemos usando linearmodels que nos permite obtener directamente el estimador de variables instrumentales y su error estándar y p-valor.

In [ ]:
# Estimamos "de una" usando linearmodels

formula = 'log_salario ~ 1 + [educ ~ t4]'
iv = IV2SLS.from_formula(formula, data = ak).fit()

param = iv.params["educ"]
se = iv.std_errors["educ"]
p_val = iv.pvalues["educ"]

print(f"Parameter: {param}")
print(f"SE:  {se}")
print(f"95 CI: {(-1.96*se,1.96*se) + param}")
print(f"P-value: {p_val}")

Vemos que la educación tiene un efecto positivo en los salarios. Un año más de educación los incrementa en un 7%. El efecto es significativo, mirar el intervalo de confianza y p-valor.

### LATE

El estimador de VI no estima el ATE si no un efecto local o un _Local Average Treatment Effect_ o LATE (sí, siguen apareciendo siglas). ¿Por qué? Porque sólo podemos capturar el efecto para aquellos donde efectivamente el tratamiento varía cuando varía el instrumento. Es decir, solamente para aquellos que tienen más educación _porque_ nacieron en el último trimestre del año. Esto hará más sentido a continuación.

### Experimentos como instrumentos, _cumplidores_ y _no-cumplidores_

En verdad, encontrar instrumentos en variables observacionales como el de Angrist y Krueguer es realmente complicado, en el sentido de realmente poder afirmar que se cumple el supuesto de validez. Ahora bien, variables instrumentales tiene un uso súper extendido como un método combinable con experimentos y sorteos, para solucionar un problema muuuy común que tienen los experimentos: el cumplimiento parcial o _partial compliance_.

Muchas veces las personas no cumplen con el tratamiento asignado. Podemos separar a las personas en 4 grupos:

* Cumplidores (_compliers_): si les toca tratamiento cumplen con el tratamiento, si les toca control cumplen con control.
* Los que siempre se tratan (_always takers_): no importa que les toque, se van a tratar.
* Los que nunca se trata (_never takers_): no importa que les toque, nunca se tratan.
* Desafiantes (_defiers_): hacen lo contrario a lo que les toca. Asumiremos que no los hay.

Esto implica que la asignación aleatoria no elimina completamente el problema de selección, hay una parte de las personas tratadas que eligen tratarse y eso probablemente significa que tienen características distintas a las que cumple o a las que eligen no hacerlo que pueden influir sobre el resultado.

Se generan dos variables que son diferentes entre sí:
* Aquellos que por sorteo o asignación aleatoria deberían haberse tratado (aquellos con "intención de ser tratados" o "_intention to treat_"). Es decir, lo que les tocó en el sorteo. ASIGNACION
* Los que efectivamente reciben tratamiento. Y esta es la variable que interesa medir. TRATAM EFECTIVO


#### _La colimba y el crimen_

Un ejemplo de Argentina con el que van a trabajar en la práctica es el trabajo de [Galiani, Rossi, & Schargrodsky (2011). Conscription and crime...](https://www.povertyactionlab.org/sites/default/files/research-paper/227_348%20Military%20conscription%20and%20crime%20in%20Argentina%20AER%20Apr%202011.pdf). Los autores quieren ver si ir a la colimba aumenta las chances de cometer un crimen luego. Para poder identificar ese efecto aprovechan los sorteos anuales que se hacían para determinar quiénes debían cumplir con el servicio militar. Primero, se asignaba un número para cada grupo de personas de la cohorte correspondiente a ese año que tengan el DNI terminado en los mismos tres dígitos. Más tarde se anunciaba un número de corte y aquellos con número por encima debían hacer el servicio militar. Además, debían pasar un examen médico para poder hacerlo.

**Ejercicio de lectura activa**: si se sorteaba la participación en el servicio militar, ¿cuál es el problema de regresar directamente el crimen en una variable indicadora de si la persona participa o no en el servicio militar? 

**Respuesta**. 

El problema es que no todas las personas hacen lo que el sorteo indica. Para empezar, hay personas que deciden participar del servicio militar independientemente del sorteo (_always takers_). En segundo lugar, los autores muestran en su trabajo que los datos sugieren que muchos hacían "trampa" en el examen médico para intentar zafar del servicio militar (_never takers_). Cuanto más cerca del corte, más enfermedades se registraban. 

Es probable que las personas que decidan por motus propia participar del servicio militar tengan características distintas a las personas que quieren evitar el servicio militar de cualquier modo. , Entonces, es probable que el promedio de las características de los tratados (los que _efectivamente_ van al servicio militar) sean distintas al promedio de las características de los no tratados (los que no van).

Entonces, podemos usar **el sorteo para instrumentar el tratamiento**, es decir, salir sorteado con número bajo como instrumento de participar en el servicio militar.

El sorteo cumple con ambas condiciones para ser instrumento:
* Relevancia: si en efecto buena parte de los sorteados son cumplidores, salir sorteado estará positivamente correlacionado con asistir al servicio militar. 
* Validez: salir sorteado sólo afecta al crimen posterior a través de la participación en el servicio militar. Como la asignación es aleatoria, lo único que cambia en promedio entre los sorteados y no sorteados es tener que hacer el servicio militar, de modo que sólo debería afectar al crimen por esa vía.

Entonces, si usamos el sorteo como instrumento de asistir al servicio militar:

* La **forma reducida** estima la **intención de ser tratado** (o ITT, por _intention to treat_) y es el **umbral inferior** del efecto de tratamiento. ¿Por qué el umbral inferior? Si todos los que salen sorteados son cumplidores,entonces la estimación de la primera etapa será 1 y la forma reducida estima el efecto de tratamiento. Si solamente algunos son cumplidores, entonces esa estimación será menor a 1, y por ende el efecto de tratamiento será mayor a la forma areducida. Si cuando algunos sorteados se tratan hay efecto, ¡mayor sería si todos se trataran!

* Variables instrumentales identifica el LATE: el impacto del tratamiento para el **subconjunto de cumplidores**, es decir para los que se tratan por haber sido asignados a tratamiento y no por otro motivo, como los _always takers_ o _never takers_. El sorteo no nos permite ver qué sucede con ellos.

* ¿Cuál es el efecto causal de la colimba en el crimen? Resolver la práctica para enterarse.


## **Diferencias en diferencias (DID)**

Podemos tener distintos tipos de datos, según la fuente de **variabilidad en el tratamiento**.

* Tratamientos que varían en una sóla dimensión
  * Datos de corte transversal (_cross-sectional_): algunos individuos son tratados y otros no, pero los observo en un sólo momento del tiempo - el tratamiento varía entre individuos.
  * Datos de serie de tiempo (_time series_): todos los individuos son no tratados y en el mismo momento pasan a ser tratados - el tratamiento varía en el tiempo. 

* Tratamientos que varían en dos dimensiones
  * Datos de panel (_panel data_): sólo un subgrupo de individuos pasa de no ser tratado a tratado en algún momento del tiempo - el tratamiento varía entre individuos y en el tiempo.
  

Según con qué datos contemos será el contrafácticos que podamos construir. Veremos que algunos contrafácticos son mejores que otros en el sentido de que necesitamos hacer **menos supuestos** sobre el proceso generador de datos para poder afirmar que estamos encontrando un efecto causal. Así, la estimación del efecto causal será más **creíble**.

### Dime qué datos tienes y te diré qué contrafáctico puedes

Supongamos 4 situaciones distintas de disponibilidad de datos y veamos qué podemos usar como contrafáctico de haber sido tratado, y qué tenemos que suponer que se cumple para poder identificar el efecto causal usando ese contrafactual.

Datos con los que contamos:

1. Contamos sólo con datos para los tratados post tratamiento

2. Contamos con datos para tratados y no tratados post tratamiento

3. Contamos con datos para tratados antes y después del tratamiento

4. Contamos con datos para tratados y no tratados, antes y después de que los tratados reciban tratamiento 

<center>
<img src="imgs/img2_datavariab.png" style="margin:auto" height=200/>
</center>


¿Qué contrafácticos podemos construir y qué tiene que cumplirse para poder identificar el efecto causal?


1. Sólo tenemos datos de tratados post tratamiento
  *  No hay variabilidad. 
  * No se puede construir un contrafáctico.

2. Tenemos datos de tratados y controles, post tratamiento.
 * Variabilidad: corte transversal. 
 * Contrafáctico: resultado de los controles post tratamiento.
 * Supuesto de identificación: los tratados, en ausencia de tratamiento, se comportará como lo hace el grupo de control luego de tratamiento. Sesgo de selección: hubiesen sido distintos después en ausencia de tratamiento.

3. Tenemos datos de tratados antes y después de tratamiento.
  * Variabilidad: serie de tiempo. 
  * Contrafácticos: resultado de los tratados antes de recibir tratamiento. 
  * Supuesto de identificación: el resultado no hubiese cambiado entre antes y después para los tratados en ausencia de tratamiento; en el interín, lo único que pudo haber afectado al outcome es el tratamiento: no afectaron otros sucesos simultáneos al tratamiento, ni se debió a un fenómeno de regresión a la media.

4. Tenemos datos de tratados y no tratados, antes y después de tratamiento.
  * Variabilidad: panel. 
  *¡Veamos que hace **diferencias en diferencias**! 
  
 

### Diferencias en diferencias en acción

Dos formas de verlo (según qué diferencia hacemos primero):


#### _Diferencia DESPUES - diferencia ANTES_

Notemos que tenemos $Y_{it}$ (resultados para distintos individuos $i$ en distintos momentos del tiempo $t$.


$$\beta^{DID} = [(Y_{TRATADOS, \; DESPUES}) - (Y_{CONTROLES, \; DESPUES})] \; \; - \; \; [(Y_{TRATADOS, \; ANTES}) - (Y_{CONTROLES, \; ANTES})]$$


* **Contrafáctico: resultado de los controles post tratamiento + diferencias preexistentes entre tratados y controles**


<center>
<img src="imgs/img3_did1.png" style="margin:auto" height=200/>
</center>


* **Supuesto de identificacion:  que las diferencias entre tratados y controles se mantienen constantes luego del tratamiento ("SUPUESTO DE TENDENCIAS PARALELAS")**
  * Se suelen **comparar las tendencias previas** para ver cuán creíble es este supuesto de identificación. Si son paralelas, es más creíble suponer que hubieran seguido siéndolo. Pero aún en el caso de que hubieran sido paralelas, ¡sigue siendo un supuesto!
  * Estamos suponiendo que **no hay inobservables que varían de forma colineal con el tratamiento**, shocks heterogéneos entre grupos simultáneos al tratamiento, algo en el momento de tratamiento que afecte de forma distinta a tratados y controles. _Nota_: inobservables, porque si no controlamos por esos shocks y solucionado el tema :)
  * Sesgo de selección: si hay una **variable confusora que varía en las mismas dos dimensiones que el tratamiento**, entre tratados y no tratados y en el tiempo (y que es inobservable y no podemos controlar por ella). En cambio, diferencias en diferencias controla por todo inobservable que varía sólo entre tratados y no tratados, o sólo en el tiempo.

#### _Diferencia TRATADOS - diferencia CONTROLES_





$$\beta^{DID} = [(Y_{TRATADOS, \; DESPUES}) - (Y_{TRATADOS, \; ANTES})] \; \; - \; \; [(Y_{CONTROLES, \; DESPUES}) - (Y_{CONTROLES, \; ANTES})]$$


* **Contrafáctico: resultado de los controles post tratamiento + diferencias preexistentes entre tratados y controles**

* **Supuesto de identificación: de no haber sido tratadas, en los individuos tratados el resultado hubiera variado lo mismo que varió en las ciudades de control ("SUPUESTO DE TENDENCIAS PARALELAS")**


<center>
<img src="imgs/img4_did2.png" style="margin:auto" height=200/>
</center>

### Carteles publicitarios en Brasil

Veamos un ejemplo con datos de [Matheus Facure en su libro _Causal Inference for the Brave and True_](https://matheusfacure.github.io/python-causality-handbook/13-Difference-in-Differences.html), donde quieren ver los efectos de una campaña publicitaria que implementaron en Brasil en los ingresos de la empresa (como la intervención real es confidencial, estos datos no son los reales, pero son similares). 

Durante el mes de junio se expuso un cartel publicitario en la ciudad de Porto Alegre. Se tienen datos de los ingresos provenientes de las ventas en esa ciudad en los meses de mayo y julio (es decir, antes y después de la intervención). Como control se cuentan con datos de la ciudad de Florianópolis, donde no se expuso ningún cartel, también para mayo y julio.

In [ ]:
brasil = pd.read_csv("https://storage.googleapis.com/humai-datasets/causalidad/brasil.txt")
brasil.head()


* **Tratados versus controles**

In [ ]:
tpost = brasil.ingresos[(brasil.porto == 1) & (brasil.jul ==1)].mean() 
cpost = brasil.ingresos[(brasil.porto == 0) & (brasil.jul ==1)].mean() 
print("Ingresos promedio en Porto Alegre en julio: " + str(tpost))
print("Ingresos promedio en Florianópolis en julio: " + str(cpost))
print("Diferencia: " + str(tpost - cpost))

**Ejercicio de lectura activa**. ¿Esto permite identificar el efecto causal? ¿Cuándo sí y cuándo no?


* **Antes versus despues**


In [ ]:
tpre = brasil.ingresos[(brasil.porto == 1) & (brasil.jul ==0)].mean() 
print("Ingresos promedio en Porto Alegre en julio: " + str(tpost))
print("Ingresos promedio en Porto Alegre en mayo: " + str(tpre))
print("Diferencia: " + str(tpost - tpre))

**Ejercicio de lectura activa**. ¿Esto permite identificar el efecto causal? ¿Cuándo sí y cuándo no?


* **Diferencias en diferencias**

In [ ]:
cpre = brasil.ingresos[(brasil.porto == 0) & (brasil.jul ==0)].mean() 

print("Diferencia en diferencia (dif. después - dif. antes)): " + str(  (tpost - cpost) - (tpre - cpre)  ))
print("Diferencia en diferencia (dif. tratados - dif. controles)): " + str(  (tpost - tpre) - (cpost-  cpre)  ))

**Ejercicio de lectura activa**. ¿Esto permite identificar el efecto causal? ¿Cuándo sí y cuándo no?

Grafiquemos el contrafactual construido:

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(["May", "Jul"], [cpre, cpost], label="CONTROL", lw=2)
plt.plot(["May", "Jul"], [tpre, tpost], label="TRATAMIENTO", lw=2)

plt.plot(["May", "Jul"], [tpre, tpre + (cpost - cpre)],
         label="Contrafáctico", lw=2, color="C2", ls="-.")

plt.legend();

Supongamos que tenemos datos para meses anteriores a mayo y observamos lo siguiente en las tendencias previas:

In [ ]:
plt.figure(figsize=(10,5))
x = ["Ene", "Mar", "May", "Jul"]

plt.plot(x, [120, 150, cpre,  cpost], label="FL", lw=2)
plt.plot(x, [60, 50, tpre, tpost], label="POA", lw=2)

plt.plot(["May", "Jul"], [tpre, tpre+(cpost-cpre)], label="Contrafáctico", lw=2, color="C2", ls="-.")

plt.legend();

**Ejercicio de lectura activa**. ¿Es razonable pensar que en este caso diferencias en diferencias funciona bien?

### Estimar DID: Mínimos Cuadrados con Variables Binarias

En nuestro ejemplo, el efecto estimado por diferencias en diferencias es bastante pequeño. Nos gustaría poder calcular los errores estándar y los p-valores de esa estimación para ver si el efecto es significativamente distinto de 0. Para eso, podemos estimar diferencias en diferencias como una regresión lineal de la variable de resultado en:
* Una variable binaria que indica el período de tiempo $t_t$ (sólo varía en el tiempo!)
* Una variable binaria que indica si una persona tratamiento o control $d_i$ (sólo varía entre individuos!)
* La interacción entre ambas, que será nuestra variable de tratamiento: $T_{it} = t_t * d_i $ será 1 para los tratados post tratamiento 

|$T_{it}$|Antes ($t_t = 0$)|Después ($t_t = 1$)|
|---|---|---|
|Tratados ($d_i = 1$)|0|1|
|Controles ($d_i = 0$)|0|0|

Es decir, estimamos el siguiente modelo:

$$Y_{it} = \beta_0 + \beta_1 T_{it} + \beta_2 t_t + \beta_3 d_i + u_{it}$$


El estimador de diferencias en diferencias será $\hat{\beta_1}$


Notemos que estamos controlando:
* Por todas las variables (observables e inobservables) que son distintas entre tratados y no tratados pero constantes en el tiempo (al controlar por $d_i$)
* Por todas las variables (observables e inobservables) que varían en el mismo momento que el tratamiento pero de la misma forma para todos los individuos (al controlar por $t_t$)

Y que las confusoras son todas las variables correlacionadas con $T_{it}$ que quedan en $u_{it}$, es decir ¡que varían en ambas dimensiones (entre tratados y no tratados y simultáneamente al tratamiento!

----
_Derivación_. ¿Por qué podemos estimar DID de esta manera?

Habíamos visto que:

 $\beta^{DID} = [(Y_{TRATADOS, \; DESPUES}) - (Y_{TRATADOS, \; ANTES})] \; \; - \; \; [(Y_{CONTROLES, \; DESPUES}) - (Y_{CONTROLES, \; ANTES})]$

Reemplacemos $Y_{it}$ en cada uno de esos cuatro casos en el modelo lineal a estimar:
* $ Y_{TRATADOS, \; DESPUES} = \beta_0 + \beta_1 1 + \beta_2 1 + \beta_3 1 + u_{it} = \beta_0 + \beta_1  + \beta_2 + \beta_3 + u_{it} $
* $ Y_{TRATADOS, \; ANTES} = \beta_0 + \beta_1 0 + \beta_2 0 + \beta_3 1 + u_{it} =  \beta_0 + \beta_3 + u_{it} $
* $ Y_{CONTROLES, \; DESPUES} = \beta_0 + \beta_1 0 + \beta_2 1 + \beta_3 0 + u_{it} =  \beta_0 + \beta_2 + u_{it}  $
* $ Y_{CONTROLES, \; ANTES} = \beta_0 + \beta_1 0 + \beta_2 0 + \beta_3 0 + u_{it} = \beta_0 + u_{it}$

Entonces, haciendo las restas:
$$\beta^{DID} = \beta_1$$

----

Estimemos ese modelo con los datos de nuestro ejemplo de Brasil:

In [ ]:
smf.ols('ingresos ~ porto + jul + porto*jul', data=brasil).fit().summary().tables[1]

## Muchos métodos más...

### Efectos fijos

Requiere datos de panel. 

Similar a diferencias en diferencias, lo que hace es:
* Introducir variables binarias de momento en el tiempo para controlar por todas las características observables e inobservables que sólo varían en el tiempo pero sean constantes entre individuos (efectos fijos de tiempo).
* Introducir variables binarias por individuo para controlar por todas las características observables e inobservables que sólo varían entre individuos pero son constantes en el tiempo (efectos fijos por individuo).

Supuesto de identificación: no hay variables inobservables que varían en el tiempo y entre individuos correlacionadas con tratamiento y resultado.

### Control sintético

Requiere datos de panel.

Variante de diferencias en diferencias: construye un sintético que funciona a modo de control "promediando" individuos de modo que la evolución previa en cuanto al resultado sea iual a la de la unidad tratada.

Supuesto de identificacion: en ausencia de tratamiento, el tratado hubiese evolucionado igual que el sintético.

### Regresión discontinua
Alcanza con datos en una dimensión pero deben ser series de tiempo de alta frecuencia o datos de corte tranversal donde hay una regla de asignación a tratamientos por umbral.

El grupo de control son los individuos _justo_ debajo del umbral o _justo_ antes de que se implemente el tratamiento.

La idea detrás es que en el entorno del tratamiento los individuos de tratamiento y ontrol los individuos son muy parecidos. Supuesto de identificación: en ausencia de tratamiento debería haber una relación continua (muy poca diferencia) en el entorno del tratamiento. Las discontinuidades son atribuibles al tratamiento.

Se puede estimar solo un efecto LOCAL, es válido en el entorno del tratamiento.



<center>
<img src="imgs/img5_regdisc.png" style="margin:auto" height=200/>
</center>

## Algunas conclusiones: qué datos $→$ qué hacemos

Para poder afirmar que encontramos el efecto de X en Y, tenemos que **suponer** que se cumple el supuesto de independencia. Cada método busca que ese supuesto se cumpla "más creíblemente" que comparaciones "así no más" entre grupos, que haga difícil pensar que las correlaciones que obseramos sean espurias, o que omitimos alguna variable confusora. A eso que tenemos que suponer sí o sí que se está cumpliendo pero **no podemos verificar empíricamente** lo lllamamos **supuesto de identificación**.


<center>
<img src="imgs/img6_mapdatmet.png" style="margin:auto" height=200/>
</center>


## Comentarios sobre la relación entre la inferencia causal y el aprendizaje automático


*   **Predecir versus identificar**. El objetivo de la mayoría de los modelos de aprendizaje automático es **predecir**, es decir, estimar la variable dependiente $Y$. Para eso, no necesitamos identificar bien ningún efecto causal. Las correlaciones (incluso las espúrias!) pueden ser muy útiles para predecir. Para predecir, no importa el coeficiente de cada variable que incluís en la regresión, ni qué pasa cuando cambia **una variable en particular**, si no el efecto de todas las variables combinadas. Entonces, volviendo a la simulación de la primera clase, no interesa si el coeficiente de fumar está captando también el hecho de que los fumadores son o no más jóvenes. Además, la importancia que se le da a la **insesgadez** al estimar los coeficientes está relacionada con identificar bien un efecto causal, pero para predecir bien nos interesa un buen Error Cuadrático Medio, y estamos dispuestos a resignar sesgo si eso mejora lo suficiente la varianza (como sucede el Lasso o Ridge).


*   **Aportes bidireccionales aprendizaje automático - inferencia causal**

### Referencias

Para repasar: 
* **Angrist & Pischke (2015), Dominar la econometría**. Capítulos 3, 4 y 5.
* **Neal (2020), Introduction to causal inference**.  Capítulos 9 y 10 (en inglés)
* **Causal Inference for the brave and true**. Capítulos 8, 9 y 13.

Para profundizar:
* Trabajos académicos mencionados en la clase sobre Variables Instrumentales (en inglés):
  * El famosísimo y pionero: [Joshua D. Angrist, Alan B. Krueger. 1991. "Does Compulsory School Attendance Affect Schooling and Earnings?" The Quarterly Journal of Economics, 106(4), 979-1014](https://www.jstor.org/stable/2937954) 
  * Un clásico en la literatura sobre desarrollo económico: [Acemoglu, Daron, Simon Johnson, and James A. Robinson. 2001. "The Colonial Origins of Comparative Development: An Empirical Investigation." American Economic Review, 91 (5): 1369-1401](https://www.jstor.org/stable/2677930) 
  * Uno interesantísimo (y entretenido) con datos de Argentina: la colimba y sus efectos en el crimen en [Galiani, S., Rossi, M. A., & Schargrodsky, E. (2011). Conscription and crime: Evidence from the Argentine draft lottery. American Economic Journal: Applied Economics, 3(2), 119-36.](https://www.povertyactionlab.org/sites/default/files/research-paper/227_348%20Military%20conscription%20and%20crime%20in%20Argentina%20AER%20Apr%202011.pdf)
* [Nota sobre el uso de diferencias en diferencias en evaluaciones de impacto](https://www.caf.com/es/actualidad/noticias/2020/02/nota-metodologica-como-implementar-diferencias-en-diferencias-para-medir-impacto/). Cita varios ejemplos, entre ellos el que usarán en la parte práctica [Di Tella, R., & Schargrodsky, E. (2004). Do police reduce crime? Estimates using the allocation of police forces after a terrorist attack. American Economic Review, 94(1), 115-133.](https://www.aeaweb.org/articles?id=10.1257/000282804322970733)

* Otros métodos: otros capítulos en los mismos libros mencionados en "Para repasar".

* Ejemplos de aportes del aprendizaje automático a la inferencia causal (en inglés): [para estimar efectos heterogéneos de tratamiento](https://escholarship.org/uc/item/3fj9z5h0)), [para asignar tratamiéntos de forma óptima](https://www.nber.org/papers/w28726), [para contruir contrafactuales](https://www.nber.org/papers/w29237).